# MLofi: Predictions

## Setup

In [1]:
model_type = 'gru'  # possible values ['srnn', 'lstm', 'gru', 'cnn']

### Hyperparameters

In [2]:
sequence_length = 30

### Imports

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate, Activation
from tensorflow.keras.layers import SimpleRNN, GRU

from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, GlobalMaxPool1D


from music21 import *

import numpy as np

import os
import re

### Mount Google Drive to access the dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def create_path(relative_path):
  return f'/content/drive/My Drive/MLofi/{relative_path}'

In [6]:
model_path = create_path(f'models/{model_type}.hdf5')
model_weights_path = create_path('weights/weights.hdf5')
path_to_unique_notes = create_path('output/data/unique_notes.txt')

### Read the note, chord and rest collection

In [7]:
with open(path_to_unique_notes, "r") as f:
  unique_notes = f.read().split("\n")
note_to_index = dict([(note, index) for index, note in enumerate(unique_notes)])
unique_note_count = len(unique_notes)

## Defining the model

In [8]:
model = keras.models.load_model(model_path)

In [9]:
model.summary()

Model: "MLofi-gru"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 30, 249)]    0           []                               
                                                                                                  
 gru_4 (GRU)                    (None, 30, 700)      1997100     ['input_2[0][0]']                
                                                                                                  
 dropout_3 (Dropout)            (None, 30, 700)      0           ['gru_4[0][0]']                  
                                                                                                  
 concatenate_3 (Concatenate)    (None, 30, 949)      0           ['input_2[0][0]',                
                                                                  'dropout_3[0][0]']      

## Predictions

In [10]:
def predict_from_seed(seed, note_count=500):
  pattern = list(seed)
  prediction = []

  for note_index in range(500):
    if model_type == 'cnn':
      p = model.predict(np.reshape(pattern, (1, sequence_length)), verbose=0)
      p = p[0]
    else:
      x = keras.utils.to_categorical(np.reshape(pattern, (1, sequence_length, 1)), num_classes=unique_note_count)
      p = model.predict(x, verbose=0)
      p = p[0]

    i = np.random.choice(unique_note_count, p=p)
    prediction.append(unique_notes[i])

    pattern.append(i)
    pattern = pattern[1:]
  return prediction

In [11]:
chord_pattern = re.compile(r'\.')
rest_pattern = re.compile(r'Rest')
note_pattern = re.compile(r'[A-Z]')

def process_prediction(prediction):
  s = stream.Stream()
  for element in prediction:
    if element == 'Padding':
      continue
    if rest_pattern.search(element) is not None:
      s.append(note.Rest(element))
    elif chord_pattern.search(element) is not None:
      s.append(chord.Chord([int(x) for x in element.split('.')]))
    elif note_pattern.search(element) is not None:
      s.append(note.Note(element))
  return s

### Random pattern

In [12]:
seed = np.random.randint(low=0, high=unique_note_count, size=sequence_length).tolist()
prediction = predict_from_seed(seed)
s = process_prediction(prediction)
s.write('midi', fp=create_path(f'output/test/06-01-2022-01-{model_type}.mid'))

'/content/drive/My Drive/MLofi/output/test/06-01-2022-01-gru.mid'